In [1]:
vocab = ["<pad>", 'El', 'perro', 'persigue', 'al', 'gato', 'y', 'se', 'lo', 'come', "<end>"]

def text_to_tokens(text, vocab):
    tokens = text.split(" ")
    return [vocab.index(token) if token in vocab else vocab.index("<pad>") for token in tokens]

frase = "El perro persigue al gato"
tokens = text_to_tokens(frase, vocab)
print(f"Tokens: {tokens}")

Tokens: [1, 2, 3, 4, 5]


In [ ]:
#from transformer import *
import torch
from Transformer import TransformerEncoder, TransformerDecoder

# covertimos a un tensor los tokens

In [3]:
tokens = torch.tensor(tokens, dtype=torch.long)
tokens

tensor([1, 2, 3, 4, 5])

# Parametros

In [ ]:
vocab_size = 10000
d_model = 512
num_layers = 6
num_heads = 8
d_ff = 2048
dropout = 0.1
max_seq_length = 5 # este tiene que tener el mismo tamaño de los tokens (checar tokens.shape[1] si es de una dimension, tokens.shape[1] si son dos dimensiones)
batch_size = 1

src_mask = torch.ones(batch_size, 1, max_seq_length, max_seq_length) # mascara que es una matriz de solos 1s porque no tenemos tokens con padding

In [5]:
tokens = tokens.unsqueeze(0)

In [6]:
tokens.shape

torch.Size([1, 5])

# Transformer Encoder

In [7]:
encoder = TransformerEncoder(vocab_size, d_model, num_layers, num_heads, d_ff, dropout, max_seq_length)
output_encoder = encoder(tokens, src_mask)

In [8]:
output_encoder.shape

torch.Size([1, 5, 512])

In [9]:
output_encoder

tensor([[[-0.3027, -0.4166, -0.3483,  ...,  0.2584,  0.7550,  0.5118],
         [ 2.6200,  0.5712,  0.2206,  ...,  0.3799,  1.2838,  0.5919],
         [ 1.0583, -0.8786,  0.1824,  ...,  0.9739, -0.7818,  0.4144],
         [ 0.1919,  0.7549,  0.6356,  ...,  1.3993, -0.3338, -0.1261],
         [-0.9554,  0.3679,  0.7908,  ...,  0.9905,  0.7334,  0.7674]]],
       grad_fn=<NativeLayerNormBackward0>)

# Transformer Decoder

In [15]:
tgt = torch.triu(torch.ones(4, 4) * float('-inf'), diagonal=1)
print(tgt)
print(tgt.shape)

tensor([[0., -inf, -inf, -inf],
        [0., 0., -inf, -inf],
        [0., 0., 0., -inf],
        [0., 0., 0., 0.]])
torch.Size([4, 4])


In [13]:
x = tokens[:, :-1]
print(x)

tensor([[1, 2, 3, 4]])


In [16]:
decoder = TransformerDecoder(vocab_size, d_model, num_layers, num_heads, d_ff, dropout, max_seq_length)
output_decoder = decoder(x, output_encoder, tgt)

In [17]:
print(output_decoder)

tensor([[[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]]],
       grad_fn=<NativeLayerNormBackward0>)


wow, looks just we must fix something here...